In [2]:
# Now I want write a test script to test all testcases in testcases folder
# The testcases in such format
# <name>.sy source code
# <name>.in input
# <name>.out std output

# compiler path: ./build/SYSY-compiler
# compiler command: ./build/SYSY-compiler <name>.sy
# I need you to run the compiler and get the output llvm ir code, which will be put in <name>.ll 
# If the compiler doesn't end with 0, keep a record of the trouble file name in CE_list
# If the compiler ends with 0, run the llvm ir code with lli and get the output, which will be put in <name>.outq
# Compare it with the std output <name>.out ,if is't not that same ,keep a record of the trouble file name in WA_list
# Remember to redirect the input and output of lli

import os
import sys
import subprocess
import filecmp
import time
import difflib
import re

compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"
pass_args=["--mem2reg"]

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".sy"):
            test_list.append(os.path.join(root, file))

CE_list = []
LLI_FAIL_list = []
WA_list = []
AC_list = []
TLE_list = []
Time_Out = []
total_time=0
Bad_test = []

for test in test_list:
    if test.endswith(".sy"):
        # add test_args to list
        print("Run pass: "+test)
        compile_args=[compiler_path, test]
        for arg in pass_args:
            compile_args.append(arg)
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            print("Compiler Error："+test)
            continue
        # run lli
        ll_file = test+".ll"
        out_file = test[:-2]+"out"
        # riscv file
        # asm_file = test + "..s"
        # maybe theres no input file
        
        if not os.path.exists(test[:-2]+"in") and not os.path.exists(out_file):
            Bad_test.append(test)
            continue
        
        # Kill program run over 5s, give it a TLE
        try:
            if not os.path.exists(test[:-2]+"in"):
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                ret = subprocess.run(["lli-14","-opaque-pointers",ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=open(test[:-2]+"in").read().encode(), timeout=10)
                # ret1 = subprocess.run(["riscv64-unknown-elf-as","-o","output.o",asm_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
        except subprocess.TimeoutExpired:
            print("Timeout Error")
            TLE_list.append(test)
            print(test)
            total_time+=15
            continue
        #timer set
        try:
            if not os.path.exists(test[:-2] + "in"):
                ret = subprocess.run(["time", "lli-14", "-opaque-pointers", ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=10)
            else:
                with open(test[:-2] + "in") as input_file:
                    input_data = input_file.read().encode()
                    ret = subprocess.run(["time", "lli-14", "-opaque-pointers", ll_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE, input=input_data, timeout=10)
        
            stderr_output = ret.stderr.decode()    
            match = re.search(r'TOTAL: (\d+)H-(\d+)M-(\d+)S-(\d+)us', stderr_output)
            if match:
                hours, minutes, seconds, microseconds = map(int, match.groups())
                # 将时间转换为秒
                total_seconds = hours * 3600 + minutes * 60 + seconds + microseconds / 1e6
                # 假设total_time已经定义，并且是以秒为单位的
                total_time += total_seconds
            else:
                print("No TOTAL time found in stderr_output.")
        except subprocess.TimeoutExpired:
            continue
        # compare the output
        # Merge The reture Value and stdout
        
        dump_str=ret.stdout.decode()
        # dump_str1 = ret1.stdout.decode()
        # remove whitesspace in the end
        # dump_str=dump_str.rstrip()
        # if dump_str1 and not dump_str1.endswith('\n'):
        #     dump_str1 += "\n"
        # if not dump_str1.endswith(''):
        #     print("RISCV Test Error")
        if dump_str and not dump_str.endswith('\n'):
            dump_str += "\n"
        dump_str += str(ret.returncode) + "\n"
        std_output=open(out_file).read()
        diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
        if(len(list(diff))!=0):
            print("Wrong Answer")
            WA_list.append(test)
            print(test)
            continue
        # print("Accepted")
        AC_list.append(test)
    
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Runtime Error: Total: "+str(len(LLI_FAIL_list)))
print("Timeout Error: Total: "+str(len(TLE_list)))
print("Wrong Answer: Total: "+str(len(WA_list)))
print("TimeOut Function:"+str(len(Time_Out)))
print("Bad Test: Total: "+str(len(Bad_test)))
print("Accepted: Total: "+str(len(AC_list)))
print("TOTAL TIME:"+str(total_time))

Run pass: ./testcases/functional/86_long_code2.sy
Run pass: ./testcases/functional/82_long_func.sy
Run pass: ./testcases/functional/45_comment1.sy
Run pass: ./testcases/functional/31_while_if_test1.sy
Run pass: ./testcases/functional/65_color.sy
Run pass: ./testcases/functional/58_sort_test4.sy
Run pass: ./testcases/functional/51_short_circuit3.sy
Run pass: ./testcases/functional/93_nested_calls.sy
Run pass: ./testcases/functional/04_arr_defn3.sy
Run pass: ./testcases/functional/40_unary_op.sy
Run pass: ./testcases/functional/99_matrix_tran.sy
Run pass: ./testcases/functional/49_if_complex_expr.sy
Run pass: ./testcases/functional/32_while_if_test2.sy
Run pass: ./testcases/functional/17_div.sy
Run pass: ./testcases/functional/03_arr_defn2.sy
Run pass: ./testcases/functional/56_sort_test2.sy
Run pass: ./testcases/functional/33_while_if_test3.sy
Run pass: ./testcases/functional/18_divc.sy
Run pass: ./testcases/functional/35_op_priority1.sy
Run pass: ./testcases/functional/98_matrix_mul.sy

In [ ]:
import os
import sys
import subprocess
import filecmp
import difflib

test_folder="./testcase"

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".s"):
            test_list.append(os.path.join(root, file))

CE_list = []
AC_list = []

for test in test_list:
    if test.endswith(".s"):
        # add test_args to list
        compile_args=["riscv64-unknown-elf-as", test, "-o", "a.out"]
        try:
            ret = subprocess.run(compile_args,timeout=60)
        except subprocess.TimeoutExpired:
            Time_Out.append(test)
            continue
        if ret.returncode != 0:
            CE_list.append(test)
            print("Compiler Error："+test)
            continue
        AC_list.append(test)
    
        
print("Compiler Error: Total: "+str(len(CE_list)))
print("Accepted: Total: "+str(len(AC_list)))

In [4]:
for k in range(len(WA_list)):
    if len(WA_list) > k:
        print(WA_list[k])

./testcases/performance/recursive_call_3.sy
./testcases/performance/fft0.sy
./testcases/performance/fft2.sy
./testcases/performance/fft1.sy
./testcases/performance/matmul3.sy
./testcases/performance/recursive_call_1.sy
./testcases/performance/recursive_call_2.sy
./testcases/h_functional/35_math.sy
./testcases/functional/65_color.sy


In [ ]:
import os
import sys
import subprocess
import filecmp
import difflib

test_folder="./testcases"

# grab all the testcases

# find files recursively
test_list = []
for root, dirs, files in os.walk(test_folder):
    for file in files:
        if file.endswith(".s"):
            test_list.append(os.path.join(root, file))

CE_list = []
AC_list = []

for test in test_list:
    if test.endswith(".s"):
            os.remove(test)

In [34]:
for k in range(len(WA_list)):
    if len(WA_list) > k:
        print(WA_list[k])

./testcases/final_performance/03_sort1.sy
./testcases/final_performance/03_sort2.sy
./testcases/final_performance/03_sort3.sy
./testcases/functional/64_calculator.sy
./testcases/hidden_functional/23_json.sy
./testcases/performance/03_sort1.sy
./testcases/performance/03_sort2.sy
./testcases/performance/03_sort3.sy


In [5]:
for k in range(len(CE_list)):
    if len(CE_list) > k:
        print(CE_list[k])

./testcases/h_performance/h-3-01.sy
./testcases/h_performance/h-3-02.sy
./testcases/h_performance/h-3-03.sy


In [ ]:
for k in range(len(TLE_list)):
    if len(TLE_list) > k:
        print(TLE_list[k])

In [ ]:
import os
import shutil

current_dir = os.getcwd()
target_folder=os.path.join(current_dir,"include")
# 遍历当前目录下的所有文件和文件夹
for root, dirs, files in os.walk(current_dir):
    if root=='/mnt/e/miniC-compiler/ir/opt':
        os.makedirs(os.path.join(target_folder,"ir/opt"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/opt")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/opt"))
    if root=='/mnt/e/miniC-compiler/backend':
        os.makedirs(os.path.join(target_folder,"backend"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"backend")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"backend"))
    if root=='/mnt/e/miniC-compiler/ir/Analysis':
        os.makedirs(os.path.join(target_folder,"ir/Analysis"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"ir/Analysis")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"ir/Analysis"))
    if root=='/mnt/e/miniC-compiler/yacc': 
        os.makedirs(os.path.join(target_folder,"yacc"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp") or file.endswith(".hh"):
                if not os.path.exists(os.path.join(target_folder,"yacc")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"yacc"))
    if root=='/mnt/e/miniC-compiler/lib':
        os.makedirs(os.path.join(target_folder,"lib"),exist_ok=True)
        for file in files:
            if file.endswith(".hpp"):
                if not os.path.exists(os.path.join(target_folder,"lib")+file):
                    path=os.path.join(root,file)
                    shutil.move(path,os.path.join(target_folder,"lib"))
